# IJCNN Experiments

Roughly speaking, I will try to use the models trained on the datasets, then remove the FC layer and add other classifiers. Then I will report the result of the other classifiers on the final performance using either EER or accuracy.

In [1]:
data_path = '../data/osaka'
results_path = '../results/'

# data_path = '/content/drive/MyDrive/IJCNN/data/osaka'
# results_path = '/content/drive/MyDrive/IJCNN/results'

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cp -r "/content/drive/MyDrive/IJCNN/src" /content/
!mv src/datasets.py datasets.py
!mv src/models.py models.py
!mv src/analyze.py analyze.py
!ls

analyze.py  datasets.py  drive	models.py  sample_data	src


In [3]:
import datasets, models, analyze
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
# from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# import nevergrad as ng
import os
np.set_printoptions(precision=4)
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [5]:
def prepare_data(train_sps, val_sps, test_sps, max_cls, split, data, labels, indices_list, run_n):
    train_split = int(split[0] * max_cls)
    val_split = int(split[2] * max_cls)

    rand_indices = np.random.choice(range(max_cls), size=max_cls, replace=False)
    indices_list.append(rand_indices)
    print('Run #%d:'%run_n)
    
    all_data_rand, all_labels_rand = [data[x] for x in rand_indices], [labels[x] for x in rand_indices]
    b_data, b_labels = datasets.generate_batch(train_sps, all_data_rand[:train_split], all_labels_rand[:train_split])
    
    val_data, val_labels = datasets.generate_batch(val_sps, all_data_rand[train_split:train_split+val_split], all_labels_rand[train_split:train_split+val_split])
    ridx = np.random.choice(range(b_data[0].shape[0]), size=b_data[0].shape[0], replace=False)
    
    b_data_test, b_labels_test = datasets.generate_batch(test_sps, all_data_rand[train_split+val_split:], all_labels_rand[train_split+val_split:])

    l_input = b_data[0][ridx]
    r_input = b_data[1][ridx]
    b_labels = b_labels[ridx]

    # print(l_input[0].shape)

    l_input_val = val_data[0]
    r_input_val = val_data[1]

    l_input_test = b_data_test[0]
    r_input_test = b_data_test[1]
    return l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test

In [6]:
def load_dataset(segLen, overlap, acc_only):
    data, labels = datasets.load_segment_osaka(data_path, 
                                            [0,744], 
                                            sample_rate=100,
                                            acc_only=acc_only,
                                            segment_time=segLen, 
                                            overlapped=overlap, 
                                            overlap=overlap, 
                                            downsample=True)
    return data, labels

In [7]:
def build_model1(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(64, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [8]:
# def build_model2(input_shape, bn, reg):
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Conv1D, \
#                                         MaxPooling1D, \
#                                         AveragePooling1D, \
#                                         Dense, Flatten, Dropout, \
#                                         BatchNormalization, \
#                                         GlobalMaxPooling1D, \
#                                         GlobalAveragePooling1D, \
#                                         Activation
#     from tensorflow.keras import activations
#     import tensorflow.keras.regularizers as regularizers

#     model = Sequential()
#     if reg:
#         model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#         model.add(Conv1D(32, 3, strides=2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#     else:
#         model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape))
#         model.add(Conv1D(32, 3, strides=2, activation='relu'))
    
#     if bn:
#         model.add(BatchNormalization())

#     model.add(MaxPooling1D(pool_size=2))

#     if reg:
#         model.add(Conv1D(64, 5, strides=2, activation='tanh', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#         model.add(Conv1D(128, 5, strides=3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#     else:
#         model.add(Conv1D(64, 5, strides=2, activation='tanh'))
#         model.add(Conv1D(128, 5, strides=3, activation='relu'))
    
#     if bn:
#         model.add(BatchNormalization())

#     model.add(GlobalMaxPooling1D())
#     return model

In [9]:
# def build_model3(input_shape, bn, reg):
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Conv1D, \
#                                         MaxPooling1D, \
#                                         AveragePooling1D, \
#                                         Dense, Flatten, Dropout, \
#                                         BatchNormalization, \
#                                         GlobalMaxPooling1D, \
#                                         GlobalAveragePooling1D, \
#                                         Activation
#     from tensorflow.keras import activations
#     import tensorflow.keras.regularizers as regularizers

#     model = Sequential()
#     model.add(Conv1D(64, 5, strides=3, padding='valid', activation='relu', input_shape=input_shape))
#     if bn: model.add(BatchNormalization())
#     model.add(Conv1D(128, 3, strides=2, padding='valid', activation='relu'))
#     model.add(MaxPooling1D(pool_size=2, strides=2))
#     model.add(Conv1D(128, 2, strides=1, padding='valid', activation='tanh'))
#     model.add(MaxPooling1D(pool_size=2, strides=2))
#     model.add(GlobalMaxPooling1D())
#     return model

In [10]:
def build_model4(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(128, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(128, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [11]:
def build_model5(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(256, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(256, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [12]:
acc_only_list    = [False, True]
segLen_list      = [1, 2]
overlap_list     = [0, 0.25, 0.5, 0.75]
# models_list      = [build_model1, build_model2, build_model3]
models_list      = [build_model5]
# bn_list          = [True, False]
bn_list          = [True]
# reg_list         = [True, False]
reg_list         = [False]

# Configurations
train_sps       = 50
val_sps         = 20
test_sps        = 20
max_cls         = 744
split           = [0.8, 0.1, 0.1]
output_dropout  = 0.1
runs            = 10
np.set_printoptions(precision=4)

In [13]:
def getSiameseAcc(siamese, test_set, EER_th):
    # l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]
    # print('EER threshold:', EER_th)
    # print(np.uint8(np.squeeze(siamese.predict(test_set[0])) >= EER_th))
    # print(b_labels_test)
    return np.mean(np.uint8(np.squeeze(siamese.predict(test_set[0])) >= EER_th) == b_labels_test)

def trainSVM(siamese, training_set, test_set):
    l_input, r_input = training_set[0]
    b_labels = training_set[1]
    l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]

    # classifier = SVC(kernel='poly', degree=10, C=100)
    classifier = SVC(kernel='poly', degree=5, C=10)
    feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
    d_vect = feature_exctractor.predict([l_input, r_input])
    d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])
    
    # print(d_vect.shape)

    classifier.fit(d_vect, b_labels)
    svc_score = classifier.score(d_vect_test, b_labels_test)
    # print('SVC mean test accuracy:', svc_score*100)
    return svc_score

def trainKNN(siamese, training_set, test_set):
    l_input, r_input = training_set[0]
    b_labels = training_set[1]
    l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]

    knn = KNeighborsClassifier(n_neighbors=5)
    feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
    d_vect = feature_exctractor.predict([l_input, r_input])
    d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])

    knn.fit(d_vect, b_labels)
    test_predictions = knn.predict(d_vect_test)
    knn_score = knn.score(d_vect_test, b_labels_test)
    # print('KNN mean test accuracy:', knn_score*100)
    return knn_score

def trainANN(siamese, training_set, test_set):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout

    l_input, r_input = training_set[0]
    b_labels = training_set[1]
    l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]

    feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
    d_vect = feature_exctractor.predict([l_input, r_input])
    d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])

    classifier = Sequential()
    classifier.add(Dense(8, activation='relu', input_shape=(256,)))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(8, activation='relu'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(1, activation='sigmoid'))
    classifier.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')

    classifier.fit(d_vect, b_labels, verbose=0, epochs=20, batch_size=128)
    # test_predictions = np.uint8(np.squeeze(classifier.predict(d_vect_test) >= 0.5))
    # print(test_predictions[:10])
    ann_score = classifier.evaluate(d_vect_test, b_labels_test)[1]
    # print('ANN mean test accuracy:', ann_score*100)
    return ann_score

In [14]:
def one_run(train_sps, val_sps, test_sps, max_cls, split, data, labels, lst, run_n):
    accuracies = {}
    l_input, r_input, b_labels \
        , l_input_val, r_input_val, val_labels \
            , l_input_test, r_input_test, b_labels_test = prepare_data( train_sps,
                                                                        val_sps,
                                                                        test_sps,
                                                                        max_cls,
                                                                        split,
                                                                        data,
                                                                        labels,
                                                                        [],
                                                                        run_n)
    print('Loaded data')
    adam = Adam(learning_rate=0.001)
    earlystop = EarlyStopping(monitor='val_accuracy', patience=5)
    
    cnn = build_model5(l_input[0].shape, bn=1, reg=0)
    siamese = models.build_siamese(l_input[0].shape, cnn, output_dropout)
    siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
    print('Built model')
    if run_n == 0: 
        cnn.summary()
    print('Start training ...')
    hist = siamese.fit([l_input, r_input], 
                        b_labels, 
                        shuffle=True,
                        batch_size=64,
                        epochs=100,
                        callbacks=[earlystop],
                        validation_data=([l_input_val, r_input_val], val_labels),
                        verbose=2
                        )
    print('Finished training')
    siamese.save_weights(os.path.join(results_path, 'osaka', 'model_r{}_weights.h5'.format(run_n)))
    print('Saved model')
    FRR, FAR, EER, EER_th = analyze.ROC(siamese, [l_input_test, r_input_test], b_labels_test)
    print('Siamese EER:', EER)
    accuracies['siamese'] = getSiameseAcc(siamese, ([l_input_test, r_input_test], b_labels_test), EER_th)
    print('Siamese Accuracy:', accuracies['siamese'])
    accuracies['svm'] = trainSVM(siamese, ([l_input, r_input], b_labels), ([l_input_test, r_input_test], b_labels_test))
    print('SVM Accuracy:', accuracies['svm'])
    accuracies['ann'] = trainANN(siamese, ([l_input, r_input], b_labels), ([l_input_test, r_input_test], b_labels_test))
    print('ANN Accuracy:', accuracies['ann'])
    accuracies['knn'] = trainKNN(siamese, ([l_input, r_input], b_labels), ([l_input_test, r_input_test], b_labels_test))
    print('KNN Accuracy:', accuracies['knn'])


    return siamese, [FRR, FAR, EER, EER_th], accuracies

### Experiment 1: Train Different Classifiers

In [17]:
data, labels = load_dataset(2, 0.75, acc_only=False)
num_cls = 4 # number of classifiers
total_accs = np.zeros((runs, num_cls))

# Results of this configuration/experiment
# EERs = np.array([np.zeros(runs) for _ in range(3*2*2)])

for run_n in range(runs):
    siamese, ROC, accs = one_run(train_sps, val_sps, test_sps, max_cls, split, data, labels, [], run_n)
    for ii, c in enumerate(accs):
        total_accs[run_n, ii] = accs[c]
    
    np.save(os.path.join(results_path, 'osaka', 'classifiers.npy'), total_accs)
    
#     EERs[4 * model_index + 2*bn + reg , run_n] = EER
#     np.save(results_path + 'osaka_filters_eers2.npy', EERs)

for ii, c in enumerate(accs):
    print(total_accs[:,ii])
    print('{} accuracy: {} +/- {}'.format(c, total_accs[:,ii].mean(), total_accs[:,ii].std()))


Run #0:
Loaded data
Built model
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 98, 64)            1216      
_________________________________________________________________
batch_normalization_4 (Batch (None, 98, 64)            256       
_________________________________________________________________
activation_4 (Activation)    (None, 98, 64)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 49, 64)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 23, 256)           82176     
_________________________________________________________________
batch_normalization_5 (Batch (None, 23, 256)           1024      
_________________________________________________________________
activation_5 (Activati

Epoch 7/100
930/930 - 8s - loss: 0.0212 - accuracy: 0.9936 - val_loss: 0.0931 - val_accuracy: 0.9655
Epoch 8/100
930/930 - 8s - loss: 0.0212 - accuracy: 0.9933 - val_loss: 0.0826 - val_accuracy: 0.9760
Epoch 9/100
930/930 - 8s - loss: 0.0162 - accuracy: 0.9948 - val_loss: 0.1070 - val_accuracy: 0.9635
Epoch 10/100
930/930 - 8s - loss: 0.0156 - accuracy: 0.9952 - val_loss: 0.1085 - val_accuracy: 0.9628
Epoch 11/100
930/930 - 8s - loss: 0.0142 - accuracy: 0.9957 - val_loss: 0.0842 - val_accuracy: 0.9733
Epoch 12/100
930/930 - 8s - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.1233 - val_accuracy: 0.9628
Epoch 13/100
930/930 - 8s - loss: 0.0125 - accuracy: 0.9960 - val_loss: 0.1195 - val_accuracy: 0.9652
Finished training
Saved model
Siamese EER: 0.023333333333333334
Siamese Accuracy: 0.9766666666666667
SVM Accuracy: 0.9723333333333334
94/94 [==============================] - 1s 2ms/step - loss: 0.4699 - acc: 0.9603
ANN Accuracy: 0.9603333473205566
KNN Accuracy: 0.9753333333333334
Run #4:

Epoch 3/100
930/930 - 8s - loss: 0.0521 - accuracy: 0.9829 - val_loss: 0.1242 - val_accuracy: 0.9595
Epoch 4/100
930/930 - 8s - loss: 0.0362 - accuracy: 0.9893 - val_loss: 0.1303 - val_accuracy: 0.9608
Epoch 5/100
930/930 - 8s - loss: 0.0298 - accuracy: 0.9907 - val_loss: 0.1379 - val_accuracy: 0.9568
Epoch 6/100
930/930 - 8s - loss: 0.0256 - accuracy: 0.9924 - val_loss: 0.1396 - val_accuracy: 0.9632
Epoch 7/100
930/930 - 8s - loss: 0.0205 - accuracy: 0.9939 - val_loss: 0.1370 - val_accuracy: 0.9645
Epoch 8/100
930/930 - 7s - loss: 0.0192 - accuracy: 0.9945 - val_loss: 0.1631 - val_accuracy: 0.9541
Epoch 9/100
930/930 - 8s - loss: 0.0179 - accuracy: 0.9944 - val_loss: 0.1354 - val_accuracy: 0.9639
Epoch 10/100
930/930 - 8s - loss: 0.0143 - accuracy: 0.9959 - val_loss: 0.1701 - val_accuracy: 0.9581
Epoch 11/100
930/930 - 8s - loss: 0.0142 - accuracy: 0.9954 - val_loss: 0.1714 - val_accuracy: 0.9608
Epoch 12/100
930/930 - 8s - loss: 0.0138 - accuracy: 0.9958 - val_loss: 0.1646 - val_accu

In [ ]:
print('Siamese network direct test accuracy:')
siamese.evaluate([l_input_test, r_input_test], b_labels_test)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM

print('Siamese network direct test accuracy:')
siamese.evaluate([l_input_test, r_input_test], b_labels_test)

print('==========SVM==========')
classifier = SVC(kernel='poly', degree=10, C=100)
feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
d_vect = feature_exctractor.predict([l_input, r_input])
d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])
print(d_vect.shape)

classifier.fit(d_vect, b_labels)
svc_score = classifier.score(d_vect_test, b_labels_test)
print('SVC mean test accuracy:', svc_score*100)

pca = PCA(2)
colors = ['#800000',
          '#bfef45']
plt.figure(figsize=(16,6))
plt.subplot(121)
plt.title('Ground Truth')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
plt.subplot(122)
plt.title('Prediction')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in classifier.predict(d_vect_test)], alpha=0.2)
plt.show()


# # print('==========OneClassSVM==========')

# # outlier_detector = OneClassSVM(kernel='rbf')
# # outlier_detector.fit(d_vect, b_labels)
# # test_predictions = np.uint8(outlier_detector.predict(d_vect_test) > 0)
# # svc_score = (test_predictions == b_labels_test).mean() * 100
# # print('OneClassSVM mean test accuracy:', svc_score)

# # pca = PCA(2)
# # colors = ['#800000',
# #           '#bfef45']
# # plt.figure(figsize=(16,6))
# # plt.subplot(121)
# # plt.title('Ground Truth')
# # d_vect_test_2d = pca.fit_transform(d_vect_test)
# # plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
# # plt.subplot(122)
# # plt.title('Prediction')
# # d_vect_test_2d = pca.fit_transform(d_vect_test)
# # plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
# # plt.show()

# print('==========ANN==========')

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout

# classifier = Sequential()
# classifier.add(Dense(8, activation='relu', input_shape=(64,)))
# classifier.add(Dropout(0.3))
# classifier.add(Dense(8, activation='relu'))
# classifier.add(Dropout(0.3))
# classifier.add(Dense(1, activation='sigmoid'))
# classifier.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')
# classifier.fit(d_vect, b_labels, verbose=0, epochs=20, batch_size=128)
# test_predictions = np.uint8(np.squeeze(classifier.predict(d_vect_test) >= 0.5))
# # print(test_predictions[:10])
# ann_score = classifier.evaluate(d_vect_test, b_labels_test)[1]
# print('ANN mean test accuracy:', ann_score*100)

# pca = PCA(2)
# colors = ['#800000',
#           '#bfef45']
# plt.figure(figsize=(16,6))
# plt.subplot(121)
# plt.title('Ground Truth')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
# plt.subplot(122)
# plt.title('Prediction')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
# plt.show()

# print('==========KNN==========')
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(d_vect, b_labels)
# test_predictions = knn.predict(d_vect_test)
# knn_score = knn.score(d_vect_test, b_labels_test)
# print('KNN mean test accuracy:', knn_score*100)

# pca = PCA(2)
# colors = ['#800000',
#           '#bfef45']
# plt.figure(figsize=(16,6))
# plt.subplot(121)
# plt.title('Ground Truth')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
# plt.subplot(122)
# plt.title('Prediction')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
# plt.show()

print('==========GaussionMixture==========')
from sklearn.mixture import GaussianMixture

gmodel = GaussianMixture(n_components=5)
gmodel.fit(d_vect, b_labels)
test_predictions =  np.uint8((gmodel.score_samples(d_vect_test) > -55))
print(test_predictions)
gmodel_score = (test_predictions == b_labels_test).mean()
print('Gaussian Mixture mean test accuracy:', gmodel_score*100)

pca = PCA(2)
colors = ['#800000',
          '#bfef45']
plt.figure(figsize=(16,6))
plt.subplot(121)
plt.title('Ground Truth')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
plt.subplot(122)
plt.title('Prediction')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
plt.show()